<a href="https://colab.research.google.com/github/josemoti1999/sysdl_project/blob/master/snn_uci_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Iris dataset

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
import torch.nn.functional as F

In [0]:
iris = pd.read_csv('iris.csv')
print(iris.shape)
iris.sample(10)

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
90,5.5,2.6,4.4,1.2,Iris-versicolor
132,6.4,2.8,5.6,2.2,Iris-virginica
134,6.1,2.6,5.6,1.4,Iris-virginica
137,6.4,3.1,5.5,1.8,Iris-virginica
9,4.9,3.1,1.5,0.1,Iris-setosa
147,6.5,3.0,5.2,2.0,Iris-virginica
32,5.2,4.1,1.5,0.1,Iris-setosa
109,7.2,3.6,6.1,2.5,Iris-virginica
57,4.9,2.4,3.3,1.0,Iris-versicolor
28,5.2,3.4,1.4,0.2,Iris-setosa


In [0]:
mappings = {
   'Iris-setosa': 0,
   'Iris-versicolor': 1,
   'Iris-virginica': 2
}
iris['species'] = iris['species'].apply(lambda x: mappings[x])

In [0]:
iris.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
136,6.3,3.4,5.6,2.4,2
135,7.7,3.0,6.1,2.3,2
85,6.0,3.4,4.5,1.6,1
54,6.5,2.8,4.6,1.5,1
39,5.1,3.4,1.5,0.2,0


In [0]:
iris_train = iris.drop('species', axis=1)

Temporal encoding

In [0]:
def temp_encoding(min_, max_, f, a=0, b=9):
    #900 time steps
    r=max_-min_
    f_new=(((b-a)/r)*f) + (((a*max_)-(b*min_))/r)
    return f_new

In [0]:
for column in iris_train.columns:
    min_=iris[column].min()
    max_=iris[column].max()
    iris[column]=iris[column].apply(lambda x: round(temp_encoding(min_, max_, x), 2))

In [0]:
iris.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
105,8.25,3.75,8.54,7.50,2
33,3.00,8.25,0.61,0.38,0
66,3.25,3.75,5.34,5.25,1
90,3.00,2.25,5.19,4.12,1
11,1.25,5.25,0.92,0.38,0


In [0]:
X = iris.drop('species', axis=1).values
y = iris['species'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [0]:
class ANN(nn.Module):
    def __init__(self, decay_multiplier=0.9, threshold=1):
        super().__init__()

        self.decay_multiplier = decay_multiplier
        self.threshold = threshold
        self.prev_inner_1 = torch.zeros([16]).to(device)
        self.prev_outer_1 = torch.zeros([16]).to(device)
        self.prev_inner_2 = torch.zeros([12]).to(device)
        self.prev_outer_2 = torch.zeros([12]).to(device)
        self.prev_inner_3 = torch.zeros([3]).to(device)
        self.prev_outer_3 = torch.zeros([3]).to(device)

        self.fc1 = nn.Linear(in_features=4, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=12)
        self.fc3 = nn.Linear(in_features=12, out_features=3)
 
    def forward(self, x):
        for i in range(900):
            #input transformation
            x=(x==i/100).float()
            input_activation_1=self.fc1(x)
            inner_excitation_1=input_activation_1+self.prev_inner_1*self.decay_multiplier
            outer_excitation_1=(inner_excitation_1>self.threshold).float()
            inner_excitation_1=inner_excitation_1 - (inner_excitation_1*outer_excitation_1)
            self.prev_inner_1 = inner_excitation_1
            
            input_activation_2=self.fc2(outer_excitation_1)
            inner_excitation_2=input_activation_2+self.prev_inner_2*self.decay_multiplier
            outer_excitation_2=(inner_excitation_2>self.threshold).float()
            inner_excitation_2=inner_excitation_2 - (inner_excitation_2*outer_excitation_2)
            self.prev_inner_2 = inner_excitation_2

            input_activation_3=self.fc3(outer_excitation_2)
            inner_excitation_3=input_activation_3+self.prev_inner_3*self.decay_multiplier
            self.prev_inner_3 = inner_excitation_3
        
        return inner_excitation_3 

In [0]:
class GeneticAlgorithm():
    def __init__(self, model, target, data):
        self.model=model
        self.target=target
        self.data=data

    def mutate(self, population_size=20, mutation_power=0.02):
        models_list=[]
        models_list.append(self.model)
        for _ in range(population_size):
            child=copy.deepcopy(self.model)
            for value in child.parameters():
                tensor_shape=value.shape
                noise=torch.randn(tensor_shape)
                value+=mutation_power*noise
            models_list.append(child)
        return models_list

    def find_best_model(self, population_size=20, mutation_power=0.02):
        models_list=self.mutate(population_size, mutation_power)
        loss_model_dict={}
        for model in models_list:
            model=model
            output=model(self.data)
            criterion = nn.CrossEntropyLoss()
            loss=criterion(output, self.target)

            loss_model_dict[model]=loss
            for (model,loss) in sorted(loss_model_dict.items(), key=lambda x: x[1], reverse=False):
                model_return=model
                loss_return=loss
                break
        return model_return, loss_return

In [0]:
POPULATION_SIZE=100
MUTATION_POWER=0.1

In [0]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=12)
        self.output = nn.Linear(in_features=12, out_features=3)
 
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x

In [0]:
model=ANN()
criterion = nn.CrossEntropyLoss()

In [0]:
epochs = 200
loss_arr = []
with torch.no_grad():
    for i in range(epochs):
        ga=GeneticAlgorithm(model, y_train, X_train)
        model, loss = ga.find_best_model(POPULATION_SIZE, MUTATION_POWER)
        loss_arr.append(loss)

        print(f'Generation: {i} Loss: {loss}')

Generation: 0 Loss: 0.8747044205665588
Generation: 1 Loss: 0.7306843400001526
Generation: 2 Loss: 0.6197757124900818
Generation: 3 Loss: 0.5756018161773682
Generation: 4 Loss: 0.4995294213294983
Generation: 5 Loss: 0.47288158535957336
Generation: 6 Loss: 0.4597247540950775
Generation: 7 Loss: 0.42335736751556396
Generation: 8 Loss: 0.3988032042980194
Generation: 9 Loss: 0.3988032042980194
Generation: 10 Loss: 0.3318929374217987
Generation: 11 Loss: 0.2974768877029419
Generation: 12 Loss: 0.28521817922592163
Generation: 13 Loss: 0.281078964471817
Generation: 14 Loss: 0.281078964471817
Generation: 15 Loss: 0.24107986688613892
Generation: 16 Loss: 0.2352432906627655
Generation: 17 Loss: 0.23092317581176758
Generation: 18 Loss: 0.23092317581176758
Generation: 19 Loss: 0.23092317581176758
Generation: 20 Loss: 0.23092317581176758
Generation: 21 Loss: 0.23092317581176758
Generation: 22 Loss: 0.22041819989681244
Generation: 23 Loss: 0.21217523515224457
Generation: 24 Loss: 0.2073754519224167
G

In [0]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [0]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df.sample()

,Y,YHat,Correct
12,0,0,1


In [0]:
df['Correct'].sum() / len(df)

0.9666666666666667